In [1]:
import pandas as pd
import os
import spacy
import re

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

/var/folders/jr/87t7453n4pqbkl_r5lb2mb5r0000gn/T/ipykernel_98884/1999909185.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)  # or 199


In [3]:
os.getcwd()

'/Users/atsukosakurai/cs224u'

In [4]:
df = pd.read_json('WashPost_NewsAPI.json')

In [5]:
df.shape

(2495, 8)

In [6]:
df.columns

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')

In [7]:
df = df.drop(columns = ['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt'])

In [8]:
df.head()

content
0  Im trying to have a similar rhythm to what I have in the office, said Ochoa, 39, an administrator at an art business in Madrid.\r\nThe right to disconnect predated the pandemic in much of Europe. The concept, first legislated in France in 2017, limits how much employees can be made to answer phone calls and email outside working hours. But the massive shift to remote work this year and the recognition that office life may never resume as it was has Spain, Greece, Ireland and other European countries discussing how they can preserve worker protections when people are working from home.\r\nWorker advocates say the always-on nature of remote work has turned pre-pandemic conversations with employers on their heads.\r\nBefore covid, all of the emphasis of the discussion was to persuade employers that work was capable of being done from home, said Irelands Esther Lynch, the deputy general secretary of the European Trade Union Confederation, an organization that represents labor union interests in Brussels. Now with covid, we are beginning to see a growing concern among workers that they wont be able to go back to the workplace, but that the new normal will be that the employer will like the idea of working from home, mostly as a way of saving money.\r\nIn Spain, Labor Minister Yolanda Díaz said Thursday that she hoped to have measures approved by next week requiring employers to pay work-from-home expenses.\r\nRemote work is no longer part of the future, but of the present of this country, she told a local radio station.\r\nDraft legislation circulated in the Spanish media would also establish a right to a flexible schedule, while setting clear hours when workers are expected to be available. The bill is expected to state that choosing to work remotely should not mean accepting a loss in pay, job stability or opportunities for promotion.\r\nGreece, too, has drafted remote work legislation. In addition to defining work hours and addressing expenses, the Greek bill would ban the use of cameras that some employers have used to make sure their employees were actually putting in their hours; require employers to respect the private lives of remote workers; and create a remote-work division within the countrys labor compliance agency. Policymakers are still discussing how to define overtime when people work flexible schedules at home.\r\nIreland, meanwhile, is in the middle of a public consultation process to draft guidelines for remote working, part of a broader push to encourage the habit even after the pandemic is over.\r\nSetting up my new home working station, Irish Deputy Prime Minister Leo Varadkar wrote in July, shortly after he took the post, as he tweeted a photo of Dell computer boxes piled on a desk. The Govt wants remote working to become part of the new normal. If done right, the benefits will be huge; reduced business costs, better work-life balance especially for parents, less traffic, lower emissions &amp; time saved on the commute.\r\nThe European efforts to ensure work-life balance contrast with the United States, where there has been much griping about the work-at-home challenges posed by the pandemic, but not much of a policy response. The U.S. Families First Coronavirus Response Act provides for expanded family and medical leave at two-thirds salary. But that is only a temporary measure, and its excludes an estimated 68 million to 106 million private-sector workers, according to an analysis by the Center for American Progress.\r\nUnlike most European countries, the U.S. government has often had a light-touch approach to regulating relations between employers and workers, especially ones who earn a salary rather than an hourly wage. And many European societies put more emphasis on work-life balance in the first place.\r\nThats made the shift to remote work somewhat easier for Europeans. For many, unchaining from the office has been one of the least painful aspects of the pandemic.\r\nBut some workers and bu

In [9]:
df.shape

(2495, 1)

In [10]:
df.shape[0]

2495

In [11]:
nlp = spacy.load('en_core_web_sm')

/Users/atsukosakurai/opt/anaconda3/lib/python3.8/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [17]:
to_remove_1 = '\r\n'
to_remove_2 = ['(', ')', '[', ']', '{', '}']

In [20]:
for i in range(df.shape[0]):
    text = df['content'].iloc[i]
    text = text.replace(to_remove_1, '')
    df['content'].iloc[i] = text

In [22]:
for i in range(df.shape[0]):
    text = df['content'].iloc[i]
    for j in range (len(to_remove_2)):
        text = text.replace(to_remove_2[j], '')
    df['content'].iloc[i] = text

In [24]:
all_sentences = []

In [25]:
for i in range(df.shape[0]):
    text = df['content'].iloc[i]
    sentences = [i for i in nlp(text).sents]
        
    for sent in sentences:
        all_sentences.append(sent)

In [26]:
len(all_sentences)

52064

In [27]:
all_sentences[0]

Im trying to have a similar rhythm to what I have in the office, said Ochoa, 39, an administrator at an art business in Madrid.

In [ ]:
# for sent in all_sentences:
#     ent_count = []
#     sent_str = sent.text.strip()
#     sent_doc = nlp(sent_str)
#     if sent_doc.ents:
#         for ent in sent_doc.ents:
#             ent = ent.text.strip()
#             ent_count.append(ent)
    
#     if len(ent_count) < 2:
#         all_sentences.remove(sent)

In [28]:
def create_entries(doc):
    
    all_rows_in_sent = []
    
    # collect NERs, create unique ID (underscore if more than one word) and put them into pairs 
    ents_per_sent = []
    
    sent_str = doc.text.strip()
    doc = nlp(sent_str)
    
    # get all entities for a sentence 
    if doc.ents:
        for ent in doc.ents:
            # convert entity as a string
            ent_str = ent.text.strip()
            ents_per_sent.append(ent_str)
        
    # create pairs out of those entities
    all_pairs = []
    
    #print('Ents found in this sentence:', ents_per_sent)
    
    for i in range(len(ents_per_sent)-1):
        pair = []
        pair.append(ents_per_sent[i])
        pair.append(ents_per_sent[i+1])
        all_pairs.append(pair)
        

    #print('Original Sentence: ', doc.text.strip())
    #print()
    
    
    for i in range(len(all_pairs)):
        
        one_row = []
        
        stringified = doc.text.strip()
        delimiters = all_pairs[i]
        ent1 = delimiters[0]
        
        ent2 = delimiters[1]
       
        pattern = r'|'.join(delimiters)
        split_string = re.split(pattern, stringified)
        
    
        ent_id_1 = ent1.replace(' ', '_') # ent_1
        ent_id_2 = ent2.replace(' ', '_') # ent_2
        

        left = split_string[0] # left   
        mention_1 = ent1 # mention_1
        
        try:
            middle = split_string[1] # middle
        except IndexError:
            middle = 'null'
        
        mention_2 = delimiters[1] # mention_2
        right = split_string[-1] # right
        
        one_row.append(ent_id_1)
        one_row.append(ent_id_2)
        one_row.append(left)
        one_row.append(mention_1)
        one_row.append(middle)
        one_row.append(mention_2)
        one_row.append(right)
        
        # get left POS
        left_doc = nlp(left)
        left_pos_list = []
        for token in left_doc:
            token_str = token.text.strip()
            token_pos = token.pos_
            unit = token_str + '/' + token_pos
            left_pos_list.append(unit)
        left_pos = (' ').join(left_pos_list)
        one_row.append(left_pos)
        
        # get mention_1 POS
        mention_1_doc = nlp(mention_1)
        mention_1_pos_list = []
        for token in mention_1_doc:
            token_str = token.text.strip()
            token_pos = token.pos_
            unit = token_str + '/' + token_pos
            mention_1_pos_list.append(unit)
        mention_1_pos = (' ').join(mention_1_pos_list)
        one_row.append(mention_1_pos)
        
        # get middle POS
        middle_doc = nlp(middle)
        middle_pos_list = []
        for token in middle_doc:
            token_str = token.text.strip()
            token_pos = token.pos_
            unit = token_str + '/' + token_pos
            middle_pos_list.append(unit)
        middle_pos = (' ').join(middle_pos_list)
        one_row.append(middle_pos)
        
        # get mention_2 POS
        mention_2_doc = nlp(mention_2)
        mention_2_pos_list = []
        for token in mention_2_doc:
            token_str = token.text.strip()
            token_pos = token.pos_
            unit = token_str + '/' + token_pos
            mention_2_pos_list.append(unit)
        mention_2_pos = (' ').join(mention_2_pos_list)
        one_row.append(mention_2_pos)
        
        # get right POS
        right_doc = nlp(right)
        right_pos_list = []
        for token in right_doc:
            token_str = token.text.strip()
            token_pos = token.pos_
            unit = token_str + '/' + token_pos
            right_pos_list.append(unit)
        right_pos = (' ').join(right_pos_list)
        one_row.append(right_pos)
        
        # print('---- ROW STARTS HERE------')
        # print(one_row)
        # print('---------END--------------')
                       
        all_rows_in_sent.append(one_row)
    
    return all_rows_in_sent
          

In [29]:
all_entries = []

In [30]:
# does not accept strings that have parenthesis?

for sentence in all_sentences:
    from_one_sent = create_entries(sentence)
    for entry in from_one_sent:
        all_entries.append(entry)

In [31]:
len(all_entries)

68780

In [32]:
all_entries[-1]

['Guaidos',
 'Venezuela',
 'Moreno says he supports ',
 'Guaidos',
 ' attempt to bring democratic change to ',
 'Venezuela',
 '.',
 'Moreno/PROPN says/VERB he/PRON supports/VERB',
 'Guaidos/NOUN',
 '/SPACE attempt/NOUN to/PART bring/VERB democratic/ADJ change/NOUN to/ADP',
 'Venezuela/PROPN',
 './PUNCT']

In [33]:
df = pd.DataFrame(all_entries)

In [40]:
df.tail(10)

,0,1,2,3,4,5,6,7,8,9,10,11
68770,U.S.,50,"Guaido, who the",U.S.,and,50,"other nations recognize as Venezuelas rightful president, says he seeks a peaceful transition that will allow his homeland to overcome its political and humanitarian crisis.","Guaido/PROPN ,/PUNCT who/PRON the/DET",U.S./PROPN,/SPACE and/CCONJ,50/NUM,"/SPACE other/ADJ nations/NOUN recognize/VERB as/ADP Venezuelas/PROPN rightful/ADJ president/NOUN ,/PUNCT says/VERB he/PRON seeks/VERB a/DET peaceful/ADJ transition/NOUN that/DET will/AUX allow/VERB his/PRON homeland/NOUN to/PART overcome/VERB its/PRON political/ADJ and/CCONJ humanitarian/ADJ crisis/NOUN ./PUNCT"
68771,50,Venezuelas,"Guaido, who the U.S. and",50,other nations recognize as,Venezuelas,"rightful president, says he seeks a peaceful transition that will allow his homeland to overcome its political and humanitarian crisis.","Guaido/PROPN ,/PUNCT who/PRON the/DET U.S./PROPN and/CCONJ",50/NUM,/SPACE other/ADJ nations/NOUN recognize/VERB as/ADP,Venezuelas/NOUN,"/SPACE rightful/ADJ president/NOUN ,/PUNCT says/VERB he/PRON seeks/VERB a/DET peaceful/ADJ transition/NOUN that/DET will/AUX allow/VERB his/PRON homeland/NOUN to/PART overcome/VERB its/PRON political/ADJ and/CCONJ humanitarian/ADJ crisis/NOUN ./PUNCT"
68772,Last_weekend,Guaido,,Last weekend,",",Guaido,"coordinated a failed effort to bring aid from Colombia and Brazil into Venezuela, where security forces loyal to Maduro blocked the supplies.",,Last/ADJ weekend/NOUN,",/PUNCT",Guaido/NOUN,"/SPACE coordinated/VERB a/DET failed/ADJ effort/NOUN to/PART bring/VERB aid/NOUN from/ADP Colombia/PROPN and/CCONJ Brazil/PROPN into/ADP Venezuela/PROPN ,/PUNCT where/ADV security/NOUN forces/NOUN loyal/ADJ to/ADP Maduro/PROPN blocked/VERB the/DET supplies/NOUN ./PUNCT"
68773,Guaido,Colombia,"Last weekend,",Guaido,coordinated a failed effort to bring aid from,Colombia,"and Brazil into Venezuela, where security forces loyal to Maduro blocked the supplies.","Last/ADJ weekend/NOUN ,/PUNCT",Guaido/NOUN,/SPACE coordinated/VERB a/DET failed/ADJ effort/NOUN to/PART bring/VERB aid/NOUN from/ADP,Colombia/PROPN,"/SPACE and/CCONJ Brazil/PROPN into/ADP Venezuela/PROPN ,/PUNCT where/ADV security/NOUN forces/NOUN loyal/ADJ to/ADP Maduro/PROPN blocked/VERB the/DET supplies/NOUN ./PUNCT"
68774,Colombia,Brazil,"Last weekend, Guaido coordinated a failed effort to bring aid from",Colombia,and,Brazil,"into Venezuela, where security forces loyal to Maduro blocked the supplies.","Last/ADJ weekend/NOUN ,/PUNCT Guaido/PROPN coordinated/VERB a/DET failed/ADJ effort/NOUN to/PART bring/VERB aid/NOUN from/ADP",Colombia/PROPN,/SPACE and/CCONJ,Brazil/PROPN,"/SPACE into/ADP Venezuela/PROPN ,/PUNCT where/ADV security/NOUN forces/NOUN loyal/ADJ to/ADP Maduro/PROPN blocked/VERB the/DET supplies/NOUN ./PUNCT"
68775,Brazil,Venezuela,"Last weekend, Guaido coordinated a failed effort to bring aid from Colombia and",Brazil,into,Venezuela,", where security forces loyal to Maduro blocked the supplies.","Last/ADJ weekend/NOUN ,/PUNCT Guaido/PROPN coordinated/VERB a/DET failed/ADJ effort/NOUN to/PART bring/VERB aid/NOUN from/ADP Colombia/PROPN and/CCONJ",Brazil/PROPN,/SPACE into/ADP,Venezuela/PROPN,",/PUNCT where/ADV security/NOUN forces/NOUN loyal/ADJ to/ADP Maduro/PROPN blocked/VERB the/DET supplies/NOUN ./PUNCT"
68776,Venezuela,Maduro,"Last weekend, Guaido coordinated a failed effort to bring aid from Colombia and Brazil into",Venezuela,", where security forces loyal to",Maduro,blocked the supplies.,"Last/ADJ weekend/NOUN ,/PUNCT Guaido/PROPN coordinated/VERB a/DET failed/ADJ effort/NOUN to/PART bring/VERB aid/NOUN from/ADP Colombia/PROPN and/CCONJ Brazil/PROPN into/ADP",Venezuela/PROPN,",/PUNCT where/ADV security/NOUN forces/NOUN loyal/ADJ to/ADP",Maduro/PROPN,/SPACE blocked/VERB the/DET supplies/NOUN ./PUNCT
68777,Maduro,Guaidos,,Maduro,described,Guaidos,gambit as part of a U.S.-backed plot to overthrow him.,,Maduro/PROPN,/SPACE described/VERB,Guaidos/NOUN,/SPACE gambit/NOUN as/ADP part/NOUN of/ADP a

In [49]:
all_sentences[-7]

Last weekend, Guaido coordinated a failed effort to bring aid from Colombia and Brazil into Venezuela, where security forces loyal to Maduro blocked the supplies.

In [35]:
df.shape

(68780, 12)

In [39]:
df.to_csv('washington_post_test.tsv', sep="\t", index=False, header=False)